# 7. 分散分析（２要因）

## 7.1　2つの要因では?

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap7/sec1.html
- テーマ
   1. 統計量算出
       - 表として表示出来る様にフォーマットを整える

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import math

# csvからデータの読み込み
df_1 = pd.read_csv('./data/chapter7-1.csv')

# 算出した統計量を表にまとめる
mean_var_list = pd.concat([df_1.count(), df_1.mean(), df_1.var(ddof=0), df_1.std(ddof=0)], axis=1).T
mean_var_list.index = ['count', 'mean', 'var', 'std']
mean_var_list

# 表示
mean_var_list


,crispy_hot,crispy_def,normal_hot,normal_def
count,15.000000,15.000000,15.000000,15.000000
mean,79.666667,71.000000,72.666667,74.333333
var,58.222222,50.666667,56.222222,59.555556
std,7.630349,7.118052,7.498148,7.717225


## 7.2　２要因の分散分析表

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap7/sec2.html
- テーマ
   1. 統計量算出
       - 表として表示出来る様にフォーマットを整える

In [2]:
# 4列分のデータを全てまとめたデータを作成
sum_1    = pd.concat([df_1.crispy_hot, df_1.crispy_def, df_1.normal_hot, df_1.normal_def], ignore_index=True)

# さらに4つのデータフレームを作成(crispy,normal/hot,def)
df_1_crispy =  pd.concat([df_1.crispy_hot, df_1.crispy_def], ignore_index=True)
df_1_normal =  pd.concat([df_1.normal_hot, df_1.normal_def], ignore_index=True)
df_1_hot    =  pd.concat([df_1.crispy_hot, df_1.normal_hot], ignore_index=True)
df_1_def    =  pd.concat([df_1.crispy_def, df_1.normal_def], ignore_index=True)

sum_1_df = pd.DataFrame({'crispy' : [df_1_crispy.count(), df_1_crispy.mean(), df_1_crispy.var(ddof=0), df_1_crispy.std(ddof=0)],
                         'normal' : [df_1_normal.count(), df_1_normal.mean(), df_1_normal.var(ddof=0), df_1_normal.std(ddof=0)],
                         'hot'    : [df_1_hot.count(),    df_1_hot.mean(),    df_1_hot.var(ddof=0),    df_1_hot.std(ddof=0)],
                         'def'    : [df_1_def.count(),    df_1_def.mean(),    df_1_def.var(ddof=0),    df_1_def.std(ddof=0)],
                         'sum'    : [sum_1.count(),       sum_1.mean(),       sum_1.var(ddof=0),       sum_1.std(ddof=0)]},
                         index=['count', 'mean', 'var', 'std'])


# 要因1(食感)によるズレ : crispy, normal
res_crispy_1  = (sum_1_df['crispy']['mean'] - sum_1_df['sum']['mean']) ** 2 * sum_1_df['crispy']['count']
res_normal_1  = (sum_1_df['normal']['mean'] - sum_1_df['sum']['mean']) ** 2 * sum_1_df['normal']['count']
res_factor1_1 = res_crispy_1 + res_normal_1

# 要因2(味付)によるズレ : hot, def
res_hot_1     = (sum_1_df['hot']['mean'] - sum_1_df['sum']['mean']) ** 2 * sum_1_df['hot']['count']
res_def_1     = (sum_1_df['def']['mean'] - sum_1_df['sum']['mean']) ** 2 * sum_1_df['def']['count']
res_factor2_1 = res_hot_1 + res_def_1

# 交互作用によるズレ　=　各群の平均のズレ　－　要因1によるズレ　－　要因2よるズレ
# -> 交互作用によるズレ　＝　各群の平均のズレ　－　要因1によるズレ　－　要因2よるズレ
sum_sq_bt_1 = []
for i in range(0, len(df_1.columns)) :
    sum_sq_bt_1.append((df_1.ix[:,i].mean() - sum_1.mean())**2 * len(df_1.ix[:,i]))

res_factor3_1 = sum(sum_sq_bt_1) - res_factor1_1 - res_factor2_1

# 残差
sum_sq_in_1 = []
for i in range(0, len(df_1.columns)) :
    sum_sq_in_1.append(df_1.ix[:,i].var(ddof=0) * len(df_1.ix[:,i]))

res_factor4_1 = sum(sum_sq_in_1)

# 4) 自由度算出
ddof_factor1_1  = 1  # 2(crispy, normal) - 1 = 1
ddof_factor2_1  = 1  # 2(hot, def) - 1 = 1
ddof_factor3_1  = 1  # 1(ddof_factor1_1) * 1(ddof_factor2_1) = 1
ddof_factor4_1  = 56 # (15 - 1) * 4

# 残差の平均平方
fval_factor4_1 = res_factor4_1 / ddof_factor4_1

# F値を算出
fval_factor1_1 = res_factor1_1 / fval_factor4_1
fval_factor2_1 = res_factor2_1 / fval_factor4_1
fval_factor3_1 = res_factor3_1 / fval_factor4_1

# 表示
print('ddof (within)   = ', ddof_factor4_1)
print('ddof (factor1)  = ', ddof_factor1_1)
print('f_val(factor1)  = ', fval_factor1_1)
print('ddof (factor2)  = ', ddof_factor2_1)
print('f_val(factor2)  = ', fval_factor2_1)
print('ddof (factor3)  = ', ddof_factor3_1)
print('f_val(factor3)  = ', fval_factor3_1)


ddof (within)   =  56
ddof (factor1)  =  1
f_val(factor1)  =  0.837784371909
ddof (factor2)  =  1
f_val(factor2)  =  3.05341246291
ddof (factor3)  =  1
f_val(factor3)  =  6.65380811078



## 7.3　交互作用とは?

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap7/sec3.html
- テーマ
   1. 読み物オンリーページ
       - 7.2での結果から読み取れることの解説


# 以下は同じことの繰り返しなので省略


## 7.4　実践編：セットでいこう

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap7/sec4.html

## 7.9　通過テスト

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap7/sec9.html


# 7.4、7.9について
- 一応、テストデータはcsvで作成済
    - chapter7-4.csv、chapter7-9.csv)
- 取り組みたいテーマ
   1. 長いコードを短くしたい(未検討)
       - 7.2をコピーして使えば結果は出せるものの、長くて読みにくい
       - 何か簡単に短く書く方法は無いものか
           - stats.f_twoway みたいなものは無さそうだし・・・
       